In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import model_selection, metrics, datasets
from neupy import algorithms, layers, environment

In [2]:
environment.reproducible()
environment.speedup()


def load_data():
    mnist = datasets.fetch_mldata('MNIST original')
    data = mnist.data

    target_scaler = OneHotEncoder()
    target = mnist.target.reshape((-1, 1))
    target = target_scaler.fit_transform(target).todense()

    n_samples = data.shape[0]
    data = data.reshape((n_samples, 1, 28, 28))

    x_train, x_test, y_train, y_test = model_selection.train_test_split(
        data.astype(np.float32),
        target.astype(np.float32),
        train_size=(6 / 7.)
    )

    mean = x_train.mean(axis=(0, 2, 3))
    std = x_train.std(axis=(0, 2, 3))

    x_train -= mean
    x_train /= std
    x_test -= mean
    x_test /= std

    return x_train, x_test, y_train, y_test



In [3]:
network = algorithms.Adadelta(
    [
        layers.Input((1, 28, 28)),

        layers.Convolution((32, 3, 3)) > layers.BatchNorm() > layers.Relu(),
        layers.Convolution((48, 3, 3)) > layers.BatchNorm() > layers.Relu(),
        layers.MaxPooling((2, 2)),

        layers.Convolution((64, 3, 3)) > layers.BatchNorm() > layers.Relu(),
        layers.MaxPooling((2, 2)),

        layers.Reshape(),
        layers.Linear(1024) > layers.BatchNorm() > layers.Relu(),
        layers.Softmax(10),
    ],

    error='categorical_crossentropy',
    step=1.0,
    verbose=True,
    shuffle_data=True,

    reduction_freq=8,
    addons=[algorithms.StepDecay],
)
network.architecture()


Main information

[ALGORITHM] Adadelta

[OPTION] batch_size = 128
[OPTION] verbose = True
[OPTION] epoch_end_signal = None
[OPTION] show_epoch = 1
[OPTION] shuffle_data = True
[OPTION] step = 1.0
[OPTION] train_end_signal = None
[OPTION] error = categorical_crossentropy
[OPTION] addons = ['StepDecay']
[OPTION] decay = 0.95
[OPTION] epsilon = 1e-05
[OPTION] reduction_freq = 8

[THEANO] Initializing Theano variables and functions.


/Users/heim/anaconda/lib/python3.5/site-packages/neupy/layers/pooling.py:193: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  st=self.stride, padding=self.padding)
/Users/heim/anaconda/lib/python3.5/site-packages/neupy/layers/pooling.py:193: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  st=self.stride, padding=self.padding)


[THEANO] Initialization finished successfully. It took 93.97 seconds

Network's architecture

--------------------------------------------------
| #  | Input shape  | Layer Type  | Output shape |
--------------------------------------------------
| 1  | (1, 28, 28)  | Input       | (1, 28, 28)  |
| 2  | (1, 28, 28)  | Convolution | (32, 26, 26) |
| 3  | (32, 26, 26) | BatchNorm   | (32, 26, 26) |
| 4  | (32, 26, 26) | Relu        | (32, 26, 26) |
| 5  | (32, 26, 26) | Convolution | (48, 24, 24) |
| 6  | (48, 24, 24) | BatchNorm   | (48, 24, 24) |
| 7  | (48, 24, 24) | Relu        | (48, 24, 24) |
| 8  | (48, 24, 24) | MaxPooling  | (48, 12, 12) |
| 9  | (48, 12, 12) | Convolution | (64, 10, 10) |
| 10 | (64, 10, 10) | BatchNorm   | (64, 10, 10) |
| 11 | (64, 10, 10) | Relu        | (64, 10, 10) |
| 12 | (64, 10, 10) | MaxPooling  | (64, 5, 5)   |
| 13 | (64, 5, 5)   | Reshape     | 1600         |
| 14 | 1600         | Linear      | 1024         |
| 15 | 1024         | BatchNorm   | 102

In [4]:
x_train, x_test, y_train, y_test = load_data()
network.train(x_train, y_train, x_test, y_test, epochs=2)

y_predicted = network.predict(x_test).argmax(axis=1)
y_test_labels = np.asarray(y_test.argmax(axis=1)).reshape(len(y_test))


Start training

[TRAINING DATA] shapes: (60000, 1, 28, 28)
[TEST DATA] shapes: (10000, 1, 28, 28)
[TRAINING] Total epochs: 2

------------------------------------------------
| Epoch # | Train err | Valid err | Time       |
------------------------------------------------
| 1       | 0.1047    | 0.0402    | 00:08:46   |                                       
| 2       | 0.02785   | 0.0266    | 00:08:41   |                                       
------------------------------------------------



In [5]:
print(metrics.classification_report(y_test_labels, y_predicted))
score = metrics.accuracy_score(y_test_labels, y_predicted)
print("Validation accuracy: {:.2%}".format(score))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      1006
          1       1.00      1.00      1.00      1157
          2       1.00      0.99      0.99      1019
          3       0.99      1.00      0.99      1008
          4       0.99      0.99      0.99       930
          5       1.00      0.99      0.99       877
          6       0.99      0.99      0.99       997
          7       0.99      1.00      0.99      1040
          8       0.98      0.99      0.99       940
          9       0.99      0.99      0.99      1026

avg / total       0.99      0.99      0.99     10000

Validation accuracy: 99.21%
